# Cyclical epidemic dynamics
Both in models and in reality, we can observe regular cycles in
the epidemiology of infectious disease outbreaks.


In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

from summer2 import CompartmentalModel

In [ ]:
# Raw input parameters
infectious_period = 7.
r0 = 13.
life_expectancy = 70.
population = 1e5
seed = 1.

# Derived parameters
beta = r0 / infectious_period
progression = 0.333
recovery = 1. / infectious_period
mortality = 1. / life_expectancy / 365.

# Create the SEIR model with demographic processes
compartments = (
    "susceptible", 
    "latent", 
    "infectious", 
    "recovered"
)
seir_demog_model = CompartmentalModel(
    times=(0, 36500.),
    compartments=compartments,
    infectious_compartments=["infectious"],
)
seir_demog_model.set_initial_population(
    distribution={
        "susceptible": population - seed, 
        "infectious": seed,
    }
)
seir_demog_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=beta,
    source="susceptible", 
    dest="latent",
)
seir_demog_model.add_transition_flow(
    name="progression", 
    fractional_rate=progression,
    source="latent", 
    dest="infectious"
)
seir_demog_model.add_transition_flow(
    name="recovery", 
    fractional_rate=recovery, 
    source="infectious", 
    dest="recovered",
)
seir_demog_model.add_universal_death_flows(
    "universal_death",
    death_rate=mortality,
)
seir_demog_model.add_replacement_birth_flow(
    "births",
    "susceptible",
)

In [ ]:
seir_demog_model.run()

In [ ]:
outputs = seir_demog_model.get_outputs_df()
outputs.plot()

## Phase plane

In [ ]:
fig = px.line(outputs.loc[outputs.index > 100.], x="susceptible", y="infectious")
fig.show()